# <p align="center"> Get Object Store File Sizes </p>

This notebooks shows the code used to extract the sizes the files for each object key in each object.

Vibe Code - created by chatgpt:

In [ ]:
credKey = ""

In [ ]:
from pathlib import Path
import sys
sys.path.append( Path( "." ).resolve().__str__() )

import os
import pickle
import time

from xaidar.filesUtils import loadPickle, savePyObj
from xaidar.s3Utils import decryptCredentials, initialize

import boto3
from concurrent.futures import ThreadPoolExecutor, as_completed

credDic = decryptCredentials( credKey, Path( "../../credentials.enc") )
# Initialize boto3 client
s3 = initialize("XChem", credDic)

def get_object_size(bucket_name, key):
    try:
        response = s3.head_object(Bucket=bucket_name, Key=key)
        return key, response['ContentLength']  # Size in bytes
    except Exception as e:
        return key, f"Error: {e}"

def fetch_sizes_parallel(bucket_name, keys, max_workers=20):
    sizes = {}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_key = {executor.submit(get_object_size, bucket_name, key): key for key in keys}
        for future in as_completed(future_to_key):
            key, size = future.result()
            sizes[key] = size

    return sizes
for bucket_name in ["xchem", "pandda"]:
    bucket_name = "pandda"
    for frag in Path( f"../../data/s3ObjKeys/{bucket_name}" ).iterdir():
        if frag.is_file():
            keys = loadPickle( frag )[1: ]
            start = time.time()
            print("Started")
            sizes = fetch_sizes_parallel( bucket_name, keys, max_workers=40 )
            print( f"Fetched sizes in {time.time() - start} seconds")

            savePath = Path("../../data/s3Sizes").joinpath( bucket_name, f"{frag.name}_sizes.pkl" ).resolve()
            print( "Saved Path: {}".format( savePath ) )    
            savePath.parent.mkdir( parents=True, exist_ok=True )
            savePyObj( sizes, savePath ) 
            print( f"Saved {bucket_name}-{frag.name}_sizes.pkl: { time.time() - start }" )
    #         break
    #     break
    # break

    
    # keys = loadPickle( Path( "../../data/s3ObjKeys/pandda/frag1.pkl") )[1: 1000]
    # start = time.time()
    # print("Started")
    # sizes = fetch_sizes_parallel( bucket_name, keys, max_workers=20 )
    # print( f"Fetched sizes in {time.time() - start} seconds")

Object Store Names:['XChem', 'MinIO']
Credentials Associated with each Object Store: ['endpoint_url', 'access_key', 'secret_key']
Started
Fetched sizes in 6.666420936584473 seconds
Saved Path: /home/alex/Documents/xaidar/data/s3Sizes/pandda/frag30.pkl_sizes.pkl
Saved pandda-frag30.pkl_sizes.pkl: 6.668848276138306


Exploaration: Looking at Data

In [3]:
import sys
from pathlib import Path
sys.path.append( Path( "." ).resolve().__str__() )

from xaidar.filesUtils import loadPickle

fragPath = Path( "../../data/s3Sizes/xchem/frag2_sizes.pkl" )
frag = loadPickle( fragPath )



In [5]:
print( type( frag) )
print( len( frag))

<class 'dict'>
10000877


In [14]:
dictKeys = list( frag.keys() )

In [48]:
idx = 0

In [91]:

print( dictKeys[ idx ] )
print( frag[ dictKeys[idx] ] )
idx += 1

data/2015/lb13308-1/processing/old_processing_prior_to_20160520/analysis/dimple-maps/KRASQ61/best/KRASQ61-x92_1_3d-run/ini.pdb
731684
